In [0]:
%run /Workspace/Repos/psureja@toromontcat.com/imacs_engineering_hours/notebooks/utility/utility_functions

In [0]:
import unittest


In [0]:


table_names=["lh_imacs.EA_Project_ActualHours","lh_imacs.EA_Project_BudgetEACHours"]
dataframe_names = ["df_project_actualHours","df_Project_BudgetEACHours"]



class Test_load_dataset(unittest.TestCase):

    @classmethod
    def setUpClass(self):
        self.a_2_2 = load_dataset(table_names, dataframe_names)

    def test_load_dataset(self):
    #   self.assertIn('a', globals(), "'a' not in globals()")
    #   self.assertIn('b', globals(), "'b' not in globals()")
    #   self.assertIn('c', globals(), "'c' not in globals()")
    #   self.assertIn('d', globals(), "'d' not in globals()")
    #   self.assertEqual(globals()['a'], 'c', 'globals()[a] is not assigned to c' )
    #   self.assertEqual(globals()['d'], 'c', 'globals()[a] is not assigned to c' )
        pass

suite = unittest.TestLoader().loadTestsFromTestCase(Test_load_dataset)
runner = unittest.TextTestRunner(verbosity=100)
results = runner.run(suite)
print(results)


In [0]:
class Test_trim_and_lower(unittest.TestCase):

    @classmethod
    def setUp(self):
        self.df_pd = pd.DataFrame(
            {'TeSt':['AaRdVaRk', 'APPLE', 'alligator'], 
             '   test_2  ':['delta  ','   disaster', '   dilapidated   ']})
        self.df_sp = trim_and_lower(spark.createDataFrame(self.df_pd), ['TeSt', '   test_2  '])

    def test_trim_post(self):
        self.assertEqual(self.df_sp.collect()[0][1], 'delta')

    def test_trim_pre(self):
        self.assertEqual(self.df_sp.collect()[1][1], 'disaster')

    def test_trim_both(self):
        self.assertEqual(self.df_sp.collect()[2][1], 'dilapidated')

    def test_lower_mixed(self):
        self.assertEqual(self.df_sp.collect()[0][0], 'aardvark')

    def test_lower_all(self):
        self.assertEqual(self.df_sp.collect()[1][0], 'apple')

    def test_lower_none(self):
        self.assertEqual(self.df_sp.collect()[2][0], 'alligator')
    

suite = unittest.TestLoader().loadTestsFromTestCase(Test_trim_and_lower)
runner = unittest.TextTestRunner(verbosity=100)
results = runner.run(suite)
print(results)

In [0]:
class Test_format_actual_hours(unittest.TestCase):

    @classmethod
    def setUp(self):
        self.df_pd = pd.DataFrame(
            {
                'StockNo': ['a', 'b', 'c'],
                'TypeCode': ['ME', 'EE', 'PM'],
                'Hours': [8.0, 3.5, 155.7],
                'ServiceSegment': ['aa', 'bb', 'cc'],
                'ServiceDate': ['07-12-2025', '01-03-2023', '12-04-2024'], # need to check actual date format and retest when data is connected
                'Test1': ['ab', 'ba', 'ac'],
                'Test2': [123, 45, 82]
            }
        )
        self.df_sp = spark.createDataFrame(self.df_pd)

    def test_kept_columns(self):
        for col_ in ['StockNo', 'TypeCode', 'Hours','ServiceSegment','ServiceMonthYear']:
            self.assertIn(col_, format_actual_hours(self.df_sp).columns)

    def test_removed_columns(self):
        for col_ in ['Test1', 'Test2']:
            self.assertNotIn(col_, format_actual_hours(self.df_sp).columns)

    def test_month_year(self):
        self.assertEqual(format_actual_hours(self.df_sp).collect()[0][4], '2025-12')
        self.assertEqual(format_actual_hours(self.df_sp).collect()[1][4], '2023-03')
        self.assertEqual(format_actual_hours(self.df_sp).collect()[2][4], '2024-04')

    
suite = unittest.TestLoader().loadTestsFromTestCase(Test_format_actual_hours)
runner = unittest.TextTestRunner(verbosity=100)
results = runner.run(suite)
print(results)

In [0]:
class Test_format_EAC_hours(unittest.TestCase):

    @classmethod
    def setUp(self):
        self.df_pd = pd.DataFrame(
            {
                'StockNo': ['a', 'b', 'c'],
                'ProjectNo': [1, 2, 3],
                'SellPriceCAD': [8.0, 3.5, 155.7],
                'Model': ['aa', 'bb', 'cc'],
                'StockType': ['gh', 'ht', 'tg'], 
                'IndustryCode': ['rj', 'j', 'ij'], 
                'Test1': ['ab', 'ba', 'ac'],
                'Test2': [123, 45, 82]
            }
        )
        self.df_sp = spark.createDataFrame(self.df_pd)

    def test_kept_columns(self):
        for col_ in ['StockNo', 'ProjectNo','SellPriceCAD','IndustryCode','Model','Stocktype']:
            self.assertIn(col_, format_EAC_hours(self.df_sp).columns)

    def test_removed_columns(self):
        for col_ in ['Test1', 'Test2']:
            self.assertNotIn(col_, format_EAC_hours(self.df_sp).columns)


suite = unittest.TestLoader().loadTestsFromTestCase(Test_format_EAC_hours)
runner = unittest.TextTestRunner(verbosity=100)
results = runner.run(suite)
print(results)

In [0]:
class Test_join_EAC_to_actual(unittest.TestCase):

    @classmethod
    def setUp(self):
        self.df_pd_eac = pd.DataFrame(
            {
                'StockNo': ['a', 'b', 'd'],
                'ProjectNo': [1, 2, 3],
                'SellPriceCAD': [8.0, 3.5, 155.7],
                'Model': ['aa', 'bb', 'cc'],
            }
        )
        self.df_sp_eac = spark.createDataFrame(self.df_pd_eac)
        self.df_pd_act = pd.DataFrame(
            {
                'StockNo': ['a', 'b', 'c'],
                'TypeCode': ['ME', 'EE', 'PM'],
                'Hours': [8.0, 3.5, 155.7],
            }
        )
        self.df_sp_act = spark.createDataFrame(self.df_pd_act)

    def test_joined_project_no(self):
        self.assertEqual(join_EAC_to_actual(self.df_sp_act, self.df_sp_eac).collect()[0][3], 1)
        self.assertEqual(join_EAC_to_actual(self.df_sp_act, self.df_sp_eac).collect()[1][3], 2)

    def test_joined_columns(self):
        for col_ in ['StockNo', 'ProjectNo', 'SellPriceCAD', 'Model', 'TypeCode', 'Hours']:
            self.assertIn(col_, join_EAC_to_actual(self.df_sp_act, self.df_sp_eac).columns)

    def test_dropped_rows(self):
        self.assertEqual(join_EAC_to_actual(self.df_sp_act, self.df_sp_eac).count(), 2)


suite = unittest.TestLoader().loadTestsFromTestCase(Test_join_EAC_to_actual)
runner = unittest.TextTestRunner(verbosity=100)
results = runner.run(suite)
print(results)

In [0]:
class Test_clean_joined(unittest.TestCase):

    @classmethod
    def setUp(self):
        self.df_pd = pd.DataFrame(
            {
                'StockNo': ['a', 'b', 'd'],
                'ProjectNo': [1, 2, None],
                'Hours': [8.0, -3.5, 155.7]
            }
        )
        self.df_sp = spark.createDataFrame(self.df_pd)


    def test_dropped_rows(self):
        self.assertEqual(clean_joined(self.df_sp).count(), 1)
        self.assertEqual(clean_joined(self.df_sp).filter(col('hours')<0).count(), 0)
        self.assertEqual(clean_joined(self.df_sp).filter(col('hours').isNull()).count(), 0)


suite = unittest.TestLoader().loadTestsFromTestCase(Test_clean_joined)
runner = unittest.TextTestRunner(verbosity=100)
results = runner.run(suite)
print(results)

In [0]:
class Test_add_recency_ind(unittest.TestCase):

    @classmethod
    def setUp(self):
        self.df_pd = pd.DataFrame(
            {
                'StockNo': ['a', 'b', 'd'],
                'ProjectNo': [1, 2, 3],
                'SellPriceCAD': [8.0, 3.5, 155.7],
                'ServiceMonthYear': ['2019-12', '2020-01', '2021-01']
            }
        )
        self.df_sp = spark.createDataFrame(self.df_pd)

    def test_joined_columns(self):
        for col_ in ['StockNo', 'ProjectNo', 'SellPriceCAD', 'ServiceMonthYear', 'recency_ind']:
            self.assertIn(col_, add_recency_ind(self.df_sp).columns)

    def test_joined_project_no(self):
        self.assertEqual(add_recency_ind(self.df_sp).collect()[0][4], 0)
        self.assertEqual(add_recency_ind(self.df_sp).collect()[1][4], 1)
        self.assertEqual(add_recency_ind(self.df_sp).collect()[2][4], 1)


suite = unittest.TestLoader().loadTestsFromTestCase(Test_add_recency_ind)
runner = unittest.TextTestRunner(verbosity=100)
results = runner.run(suite)
print(results)

In [0]:
class Test_compress_to_project_level(unittest.TestCase):

    @classmethod
    def setUp(self):
        self.df_pd = pd.DataFrame(
            {
                'ProjectNo': [1, 2, 3, 1, 2],
                'StockNo': ['a', 'b', 'c', 'd', 'e'],
                'SellPriceCAD': [8.0, 3.5, 155.7, 7.0, -2],
                'Model': ['a', 'b', 'c', 'a', 'c'],
                'recency_ind': [1, 1, 1, 0, 1],
                'Hours': [5, 10, 15, 20, 25],
                'IndustryCode': ['a', 'b', 'c', 'b', 'a']
            }
        )
        self.df_sp = spark.createDataFrame(self.df_pd)

    def test_grouping(self):
        self.assertEqual(compress_to_project_level(self.df_sp).count(), 3)

    def test_stock_count(self):
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[0][1], 2)
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[1][1], 2)
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[2][1], 1)

    def test_distinct_model_count(self):
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[0][2], 1)
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[1][2], 2)
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[2][2], 1)

    def test_recency_ind(self):
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[0][3], 0)
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[1][3], 1)
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[2][3], 1)

    def test_hour_sum(self):
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[0][4], 25)
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[1][4], 35)
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[2][4], 15)

    def test_sell_price_sum(self):
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[0][5], 15)
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[1][5], 1.5)
        self.assertEqual(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[2][5], 155.7)

    def test_industry_code(self):
        # we don't actually care which industry code is chosen, so only checking inclusion instead of specific equality
        self.assertIn(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[0][6], {'a', 'b'})
        self.assertIn(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[1][6], {'a', 'b'})
        self.assertIn(compress_to_project_level(self.df_sp).sort('ProjectNo').collect()[2][6], {'c'})


suite = unittest.TestLoader().loadTestsFromTestCase(Test_compress_to_project_level)
runner = unittest.TextTestRunner(verbosity=100)
results = runner.run(suite)
print(results)

In [0]:
class Test_filter_outliers(unittest.TestCase):

    @classmethod
    def setUp(self):
        self.df_pd = pd.DataFrame(
            {
                'ProjectNo': [1, 2, 3, 4],
                'stock_count': [500, 501, 500, 500],
                'total_sell_price': [100000000, 100000000, 100000001, 100000000],
                'hours': [1500, 1500, 1500, 1501],
                'ServiceMonthYear': ['a', 'b', 'c', 'd']
            }
        )
        self.df_sp = spark.createDataFrame(self.df_pd)

    def test_removed_rows(self):
        self.assertEqual(filter_outliers(self.df_sp).count(), 1)

    def test_removed_values(self):
        self.assertEqual(filter_outliers(self.df_sp).collect()[0][1], 500)
        self.assertEqual(filter_outliers(self.df_sp).collect()[0][2], 100000000)
        self.assertEqual(filter_outliers(self.df_sp).collect()[0][3], 1500)



suite = unittest.TestLoader().loadTestsFromTestCase(Test_filter_outliers)
runner = unittest.TextTestRunner(verbosity=100)
results = runner.run(suite)
print(results)

In [0]:
class Test_bc_transform(unittest.TestCase):

    @classmethod
    def setUp(self):
        self.df_pd = pd.DataFrame(
            {
                'ProjectNo': [1, 2, 3, 4],
                'stock_count': [500, 2501, 5070, 500],
                'hours': [15, 150, 1500, 15000],
            }
        )
        self.df_sp = spark.createDataFrame(self.df_pd)

    def test_feature_invariance(self):
        self.assertEqual(bc_transform(self.df_sp)[0].collect()[0][0], 1)
        self.assertEqual(bc_transform(self.df_sp)[0].collect()[1][0], 2)
        self.assertEqual(bc_transform(self.df_sp)[0].collect()[2][0], 3)
        self.assertEqual(bc_transform(self.df_sp)[0].collect()[3][0], 4)

        self.assertEqual(bc_transform(self.df_sp)[0].collect()[0][1], 500)
        self.assertEqual(bc_transform(self.df_sp)[0].collect()[1][1], 2501)
        self.assertEqual(bc_transform(self.df_sp)[0].collect()[2][1], 5070)
        self.assertEqual(bc_transform(self.df_sp)[0].collect()[3][1], 500)

    def test_target_change(self):
        self.assertNotEqual(bc_transform(self.df_sp)[0].collect()[0][2], 15)
        self.assertNotEqual(bc_transform(self.df_sp)[0].collect()[1][2], 150)
        self.assertNotEqual(bc_transform(self.df_sp)[0].collect()[2][2], 1500)
        self.assertNotEqual(bc_transform(self.df_sp)[0].collect()[3][2], 15000)


suite = unittest.TestLoader().loadTestsFromTestCase(Test_bc_transform)
runner = unittest.TextTestRunner(verbosity=100)
results = runner.run(suite)
print(results)